# Worksheet 08

Name:  Rohan Anand

UID: U23735730

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [17]:
import random
import numpy as np
from sklearn.cluster import KMeans


mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.489000304558442, 3.1881990847673203, 3.677752580591954, 4.998275011684966, 6.574213367221136, 5.562121291796986, 5.134669295740058, 6.169851012541715, 5.1797142187059615, 4.010088277285393]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [18]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[7.680490809215315, 7.914944611336911, 6.87079372674043, 6.767963731651916, 8.357918266895211, 7.925396538543261, 8.040964332597055, 6.704784293518003, 9.865730196407435, 6.9274886079749]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [19]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[6.9274886079749, 4.010088277285393, 9.865730196407435, 6.704784293518003, 8.040964332597055, 7.925396538543261, 8.357918266895211, 6.767963731651916, 5.1797142187059615, 6.87079372674043]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The data is created from random values from two distributions.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [20]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.010088277285393, 5.1797142187059615]
[6.9274886079749, 9.865730196407435, 6.704784293518003, 8.040964332597055, 7.925396538543261, 8.357918266895211, 6.767963731651916, 6.87079372674043]
P(C_1) = 0.2,  P(C_2) = 0.8
mean_1 = 4.5949012479956775,  mean_2 = 7.682629961791027
var_1 = 0.3420062107109875,  var_2 = 1.053924132722905


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [21]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( prob_c[0] * pdf_i[0] / prob_x )
    prob_c1_x.append( prob_c[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  6.9274886079749
probability of observing that point if it came from cluster 0 =  9.245435263107942e-11
probability of observing that point if it came from cluster 1 =  0.2928343086814163
point =  4.010088277285393
probability of observing that point if it came from cluster 0 =  0.2703672913694235
probability of observing that point if it came from cluster 1 =  0.0008736750314946848
point =  9.865730196407435
probability of observing that point if it came from cluster 0 =  3.099485417715015e-52
probability of observing that point if it came from cluster 1 =  0.044298188659475116
point =  6.704784293518003
probability of observing that point if it came from cluster 0 =  6.34793933359165e-09
probability of observing that point if it came from cluster 1 =  0.2461341662741961
point =  8.040964332597055
probability of observing that point if it came from cluster 0 =  1.048878303469228e-22
probability of observing that point if it came from cluster 1 =  0.35727158152728333
point =  7

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [22]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.17369660562331588,  P(C_2) = 0.8263033943766841
mean_1 = 4.514934068012785,  mean_2 = 7.601149335157923
var_1 = 0.3356114818681837,  var_2 = 1.2243086749384613


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [23]:
updated_prob_c0_x = [] 
updated_prob_c1_x = []  

for p, pc0_x, pc1_x in zip(data, prob_c0_x, prob_c1_x):
    updated_pc0_x = (pc0_x * prob_c[0]) / prob_x
    updated_pc1_x = (pc1_x * prob_c[1]) / prob_x

    updated_prob_c0_x.append(updated_pc0_x)
    updated_prob_c1_x.append(updated_pc1_x)

print("Updated P(C_0 | X_i) = ", updated_prob_c0_x)
print("Updated P(C_1 | X_i) = ", updated_prob_c1_x)

Updated P(C_0 | X_i) =  [6.091054671066423e-11, 0.7618500015216632, 1.3498661979404264e-51, 4.975628595148084e-09, 5.663875817246925e-23, 1.5610820603929153e-21, 3.760463746213902e-27, 1.4829451476837694e-09, 0.578562328000558, 1.936920151251657e-10]
Updated P(C_1 | X_i) =  [3.6710921812515243, 0.04684610025384731, 3.671092181541286, 3.6710921578713998, 3.671092181541286, 3.671092181541286, 3.671092181541286, 3.671092174486671, 0.9187757861242596, 3.6710921806198606]


h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [24]:
hard_assignments = [] 

for pc0_x, pc1_x in zip(updated_prob_c0_x, updated_prob_c1_x):
    if pc0_x >= pc1_x:
        hard_assignments.append(0)
    else:
        hard_assignments.append(1)

print("Hard assignments = ", hard_assignments)

Hard assignments =  [1, 0, 1, 1, 1, 1, 1, 1, 1, 1]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Disagree       |
| A  C |      Disagree       |
| A  D |      Disagree       |
| A  E |      Disagree       |
| B  C |      Disagree       |
| B  D |      Disagree       |
| B  E |      Disagree       |
| C  D |      Disagree       |
| C  E |      Disagree       |
| D  E |      Disagree       |

As the dataset becomes larger, there are more ordered pairs which means it becomes a computational burden to determine agreement for all pairs



b) Given N points, what is the formula for the number of unique pairs of points one can create?

(N * (N - 1)) / 2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Agreements is 10 

Total Disagreements is 26

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Compare points within the cluster for each clustering method